## testWaOkConversion

It simply iterates the candidates in `Variable Substitution Candidate Map`(VSCM), following that substitution rules, and check if the substitution is correct. It is dumb procedure, but it allows testing of variable prioritization technic at a lower cost than compile them.

In [1]:
import os
#import difflib

In [2]:
# Test Dataset Directory Schema

# testDirname
# - dir1
#   - Wrong answer 1
#   - Wrong answer 2
#   - Wrong ...
#   - Correct answer
# - dir2
#   - Wrong answers ...
#   - Correct answer
# - dir...

# Wrong answer should have the substring 'WA' in their filename,
# Correct answer should have the substring 'OK' in their filename.

testDirname = '/home/ubuntu/workspace/pyws/modifyCpp/pairData'

In [3]:
# Convert Test Dataset Directory into WA/OK filename list.
# Input: Directory name
# Output List Type:
#   <<WA-Filename> list * <OK-Filename> list> list
# - Filenames are absolute path strings.

def getWaOkFilenames(dirname):
    indirnames = next(os.walk(dirname))[1]
    outputList = []
    for indn in indirnames:
        indnfull = os.path.join(dirname, indn)
        fns = next(os.walk(indnfull))[2]
        #split into WA and OK
        was = [(os.path.join(indnfull, x)) for x in fns if 'WA' in x]
        oks = [(os.path.join(indnfull, x)) for x in fns if 'OK' in x]
        outputList.append((was, oks))
    return outputList

In [4]:
# Compare two string.
# For now, check if the two input strings are exactly same or not.

# input: 
#   - s1 (string list)
#   - s2 (string list)
# output: bool

def sDiff_1(s1, s2):
    if (len(s1) != len(s2)):
        return False

    for ss1, ss2 in zip(s1, s2):
        if ss1 != ss2:
            return False
    return True
        
# sDiff_2 deletes all blanks in string.
def sDiff_2(s1, s2):
    ss1 = ''.join(''.join(s1).split())
    ss2 = ''.join(''.join(s2).split())
    return ss1 == ss2

In [8]:
# input: 
#   - waFname (string) (w/a filename, include path)
#   - okFname (string) (correct answer filename, include path)
#   - tmpFilename (string) (vscm filename generated for w/a file, include path)
#   - sDiffFunc ((string list * string list) -> bool)
#       compare two strings (To be exact, it compare two string lists.)
#   - iterLimit (int) (limiting number of vscm iteration. optional. 0 or negative number for no limit.)
#   - totalTimeout (int) (represent timeout seconds. optional. 0 or negative number for no limit.)
#   - printResult (bool) (if true, print result information)

# output: multiple return values.
#   - vscm (list(dictionary)). 
#       VSCM details are located at 'readVscm' function in 'modifyCode.py'
#   - correct vscm index (int)
#   - running time (int) (seconds)
#   - # of vscm iteration (int)
#   - return code (int)
#     - -1: Undefined (this should not happened)
#     - 0: success
#     - 1: fail for all candidates
#     - 2: fail and reaches iterLimit
#     - 3: fail and reaches totalTimeout

# correct source code is not returned since the correct source code is already given.

import makeVscm
import modifyCode
import time

def solve(waFname, okFname, tmpFilename, sDiffFunc, iterLimit=0, totalTimeout=0, printResult=False):
    resultVscmIndex = -1
    resultVscmIterCount = -1
    resultCode = -1
    
    startTime = time.time()
    timeout = startTime + totalTimeout
    
    # make & read vscm
    makeVscm.genVscm(waFname, tmpFilename)
    vscm = modifyCode.readVscm(tmpFilename)
    
    # get wafile, okfile's source code.
    with open(waFname, 'r') as waf:
        with open(okFname, 'r') as okf:
            was = waf.readlines()
            oks = okf.readlines()
            
    # iterate vscm
    breakFlag = False
    for i in range(len(vscm)):
        # timeout
        if totalTimeout > 0 and time.time() > timeout:
            resultVscmIterCount = i
            resultCode = 3
            breakFlag = True
            break
        # iterLimit
        if iterLimit > 0 and i >= iterLimit:
            resultVscmIterCount = i
            resultCode = 2
            breakFlag = True
            break
        
        ms, lineind, rs = modifyCode.modCodeFull(was, vscm, i)
            
        if sDiffFunc(ms, oks):
            resultVscmIterCount = i
            resultVscmIndex = i
            resultCode = 0
            breakFlag = True
            break
    
    
    if not breakFlag:
        # fail to find appropriate variable substitute
        resultVscmIterCount = len(vscm)
        resultCode = 1
    
    resultElapsedTime = time.time() - startTime
        
    if printResult:
        print('WA filename: ' + os.path.basename(waFname))
        print('OK filename: ' + os.path.basename(okFname))
        print('vscmFilenme: ' + os.path.basename(tmpFilename))
        print('iterLimit  : ' + str(iterLimit))
        print('timeLimit  : ' + str(totalTimeout))
        if(resultCode == 0):
            print('\tStatus:\t\tSuccess')
            print('\tVscm Index:\t' + str(resultVscmIndex))
        elif(resultCode == 1):
            print('\tStatus:\t\tFail')
        elif(resultCode == 2):
            print('\tStatus:\t\tIterLimit')
        elif(resultCode == 3):
            print('\tStatus:\t\tTimeout')
            
        print('\tIterCount:\t' + str(resultVscmIterCount))
        print('\tElapsed Time:\t' + str(resultElapsedTime))
        
    return vscm, resultVscmIndex, resultElapsedTime, resultVscmIterCount, resultCode


In [9]:
# TEST once.
a = getWaOkFilenames(testDirname)
b = a[0]
_ = solve(b[0][0], b[1][0], 'tmpTestFile.vscm', sDiff_2, printResult=True)


WA filename: 59220350WA.cpp
OK filename: 59221613OK.cpp
vscmFilenme: tmpTestFile.vscm
iterLimit  : 0
timeLimit  : 0
	Status:		Success
	Vscm Index:	448
	IterCount:	448
	Elapsed Time:	1.3268530368804932


## TEST

In [10]:
# Test for all files in testDirname
TEST_FLAG = True

import itertools

if TEST_FLAG:
    testVscmFilename = 'tmpTestFile.vscm'
    testcaseCount = 0
    resultList = []
    sdiffFunction = sDiff_2
    a = getWaOkFilenames(testDirname)
    
    
    for b in a:
        c = list(itertools.product(b[0], b[1]))
        for d in c:
            testcaseCount += 1
            print('\nTESTCASE\t' + str(testcaseCount))
            resultList.append(solve(d[0], d[1], testVscmFilename, sdiffFunction, printResult=True))


TESTCASE	1
WA filename: 59220350WA.cpp
OK filename: 59221613OK.cpp
vscmFilenme: tmpTestFile.vscm
iterLimit  : 0
timeLimit  : 0
	Status:		Success
	Vscm Index:	38
	IterCount:	38
	Elapsed Time:	1.3470473289489746

TESTCASE	2
WA filename: 59198544WA.cpp
OK filename: 59198582OK.cpp
vscmFilenme: tmpTestFile.vscm
iterLimit  : 0
timeLimit  : 0
	Status:		Success
	Vscm Index:	750
	IterCount:	750
	Elapsed Time:	1.3233668804168701

TESTCASE	3
WA filename: 59175575WA.cpp
OK filename: 59176229OK.cpp
vscmFilenme: tmpTestFile.vscm
iterLimit  : 0
timeLimit  : 0
	Status:		Fail
	IterCount:	1285
	Elapsed Time:	1.3705379962921143

TESTCASE	4
WA filename: 59185559WA.cpp
OK filename: 59185600OK.cpp
vscmFilenme: tmpTestFile.vscm
iterLimit  : 0
timeLimit  : 0
	Status:		Success
	Vscm Index:	899
	IterCount:	899
	Elapsed Time:	1.4258522987365723

TESTCASE	5
WA filename: 59143819WA.cpp
OK filename: 59269894OK.cpp
vscmFilenme: tmpTestFile.vscm
iterLimit  : 0
timeLimit  : 0
	Status:		Fail
	IterCount:	110
	Elapsed T

In [1]:
from copy import deepcopy